### Server Setup

Creating a stream of data & sending it to the clients.

<br>
<br>

In [1]:
# to specify delay between two consecutive records in stream
DELAY = 0.1

In [2]:
# Importing required modules

# data streaming
import asyncio
import websockets
from time import sleep

# data processing
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
# Read csv files of merged data

# # original data
# data = pd.read_csv("../data/IoT_Garage_Door.csv", low_memory=False)

# test data for presentation purpose
data = pd.read_csv("../data/test_Weather.csv", low_memory=False)

In [4]:
# dropping NaN or NA values
data = data.dropna()

# encoding string type variables to numeric type (required for model training)
encoder=LabelEncoder()
data['type']=encoder.fit_transform(data['type'])

In [5]:
data.head()

,date,time,temperature,pressure,humidity,label,type
0,28-Apr-19,06:59:35,38.184990,1.035,99.448695,1,0
1,28-Apr-19,06:59:40,38.680906,1.035,99.448695,1,0
2,28-Apr-19,06:59:45,38.531486,1.035,99.448695,1,0
3,28-Apr-19,06:59:50,38.336073,1.035,99.448695,1,0
4,28-Apr-19,06:59:55,38.611960,1.035,99.448695,1,0


In [6]:
async def server(websocket):
    """
    Server function to stream data row-wise in a comma separated string format.
    """

    # convert a row to space(" ") separated string format
    temp = data.to_string(header=False,index=False,
                  index_names=False).split('\n')

    # now convert above row to comma(",") separated string format
    rows = [','.join(ele.split()) for ele in temp]
    
    c=0    # variable to keep track of number of records sent

    # while 'c' is less than & equal to total number of records in dataset
    # this loop is responsible for creating data stream
    while(len(data)>c):
        # send stringified row to the client
        await websocket.send(rows[c])

        # wait of acknowledgement
        ok = await websocket.recv()
        if(bool(ok)):
            c+=1    # if successfully received then increment 'c' by 1
        
        # add artificial delay to simulate a remote server client setup having communication delays due to network traffic
        sleep(DELAY)

async def main():
    async with websockets.serve(server, "localhost", 8765, ping_interval=None):
        await asyncio.Future()  # run forever

await main()

<br>
<br>
<center><b>End of File</b></center>